# Introduction to Python with ROS
In this notebook, we will go over some of the fundamental python concepts to get started with programming for Colorado RoboSub. If you have not used python notebooks before, you can run a code cell by pressing the play button, or hit `command/control + shift`. Most of you have probably come from CSCI 1300, so you have some basic programming experinece. We will not go over the basic datatypes and classes, but rather the ROS2 API.

## Fundamental Concepts

This section will describe some of the fundamental concepts needed to work with ROS, and how to implement some of that stuff in Python. For more information, please visit the [ROS2 tutorials](https://docs.ros.org/en/humble/Tutorials.html). Note also, unless you have a native Ubuntu installation, the turtlesim tutorial described in the tutorials won't work.

### Nodes

You can think of a node as a single program. They can do a variety of different things. In the context of the submarine, we have nodes that send a PWM signal to the motors to make them move, a node to take in Joystick input, a node that publishes camera data, etc. Nodes publish data over something called a *topic*. Nodes can also subscribe to topic data, such that it can be manipulated.

### Topics

You can think of topics as the pipes that transport data between programs. A robot can have multiple different nodes running at the same time, but those nodes need to communicate with each other. Described in a little more detail here, we can take our joystick data and publish it over a topic called `/joy`. With another node (a different Python program), we can *subscribe* to the topic and modify that data such that it sends the right data to a `/cmd_vel` topic (another pipe), which then sends the data to yet another node that powers the motors.

All of the nodes, when there is a change in some data, will send a *message* to each of their targeted topics.

### The most basic example (stolen from the ROS docs)

In [ ]:
# simple python publisher

import rclpy
from rclpy.node import Node

from std_msgs.msg import String


class MinimalPublisher(Node):

    def __init__(self):
        super().__init__('minimal_publisher') # node name
        self.publisher_ = self.create_publisher(String, 'topic', 10) # the topic is called "topic" - queue size is for backfill of messages
        timer_period = 0.5  # seconds
        self.timer = self.create_timer(timer_period, self.timer_callback) # every 0.5 seconds, call the timer_callback function
        self.i = 0

    def timer_callback(self):
        msg = String()
        msg.data = 'Hello World: %d' % self.i
        self.publisher_.publish(msg)
        self.get_logger().info('Publishing: "%s"' % msg.data)
        self.i += 1


def main(args=None):
    rclpy.init(args=args)

    minimal_publisher = MinimalPublisher()

    rclpy.spin(minimal_publisher)

    # Destroy the node explicitly
    # (optional - otherwise it will be done automatically
    # when the garbage collector destroys the node object)
    minimal_publisher.destroy_node()
    rclpy.shutdown()


if __name__ == '__main__':
    main()

In [ ]:
# minimal subscriber

import rclpy
from rclpy.node import Node

from std_msgs.msg import String


class MinimalSubscriber(Node):

    def __init__(self):
        super().__init__('minimal_subscriber')
        self.subscription = self.create_subscription(
            String,
            'topic',
            self.listener_callback,
            10)
        self.subscription  # prevent unused variable warning

    def listener_callback(self, msg): # upon hearing a message, call this function
        self.get_logger().info('I heard: "%s"' % msg.data)


def main(args=None):
    rclpy.init(args=args)

    minimal_subscriber = MinimalSubscriber()

    rclpy.spin(minimal_subscriber)

    # Destroy the node explicitly
    # (optional - otherwise it will be done automatically
    # when the garbage collector destroys the node object)
    minimal_subscriber.destroy_node()
    rclpy.shutdown()


if __name__ == '__main__':
    main()

### Message types

There are several different messages that we can pass over ROS. In the example above, we passed a `String` type. Notice, it's still it's own imported class (different than python). ROS has several message types that come with the standard installation. Some of our most common are listed below:
- Geometry Messages
    - Twist
    - Pose
- Standard Messages
    - String
    - Boolean

For a comprehensive list of geometry messages, look [here](https://docs.ros2.org/galactic/api/geometry_msgs/index-msg.html).

For a comprehensive list of standard messages, look [here](https://docs.ros2.org/galactic/api/std_msgs/index-msg.html).

ROS is a much more comprehensive tool, but this example serves on how to write ROS nodes. You may find the ROS commands listed in another .ipynb useful for debugging.

### Packages and Launch Files

You can think of packages as a folder containing a collection of related nodes. Python packages can be created by doing `ros2 pkg create --build-type ament_python <package_name>` in the `src` directory of the `cusub2.1` workspace. C++ packages can be created using `ros2 pkg create --build-type ament_cmake <package_name>`. ROS nodes must exist within these packages. More often than not, if you are writing task code, the tasks package has been created for you.

There is a defined structure to these packages:
```
src
└── teleoperation
    ├── launch
    │   └── teleop_launch.xml # launch file for the package (not required)
    ├── package.xml
    ├── resource # necessary for ROS filepath finding for packages
    │   └── teleoperation
    ├── setup.cfg
    ├── setup.py # configuration for launch files done here
    ├── teleoperation
    │   ├── __init__.py
    │   ├── joyListener.py
```

Generally, all of your code should be within src/pkg_name/pkg_name.

Launch files are simply a way to launch a set of nodes from a package. In most instances, we only have a singular node in each package (this will change). You will notice that you can launch an individual node from the terminal, but if you have multiple nodes that you need to run in a package, that is where the launch file comes in. Details for how to launch something are in the cheat sheet file.

### Practice Exersises
1. Create a publisher that publishes a message with your name every two seconds. Create a subscriber that prints that message using `self.get_logger().info("<msg>")`
2. Create a subscriber that takes in joystick `Twist` data from the `/joy` topic and logs when the trigger is pressed, and when the joystick is pulled all the way back.